In [1]:
# Making sure to use the updated version
%load_ext autoreload
%autoreload 2

In [99]:
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from time import sleep
from random import randint
import re

# From Postcodes

In [3]:
option = Options()
option.headless = True
driver = webdriver.Firefox(options=option)

In [30]:
driver.get('https://www.postnl.nl/en/find-an-address/')

In [194]:
def get_full_address(postcode, number):
    driver.find_element(By.ID, 'postalcode').clear()
    driver.find_element(By.ID, 'number').clear()
    
    driver.find_element(By.ID, 'postalcode').send_keys(postcode)
    driver.find_element(By.ID, 'number').send_keys(number)
    l = driver.find_element(By.XPATH, "//input[@type='submit' and @value='Zoek adres ']")
    driver.execute_script("arguments[0].click();", l)
    
    sleep(1)
    
    result = driver.find_elements(By.XPATH, "//div[@class='result']")
    ans = []
    for r in result:
        try:
            street, h_number, toeg, pc, stad = (re.compile("<p>(\w+)<span>(\d+) (.*)</span></p><p>(\w+)<span>(\w+)</span></p>").match(r.get_attribute('innerHTML')).groups())
            print(street, h_number, toeg, '\n' + pc, stad)
            ans.append(list(map(lambda x: x.upper(), (street, h_number, toeg, pc, stad))))
        except:
            print('bad address')
    
    return ans
   

How to deal with the `nummber`+`toegevolgd`?

Some examples:

In [196]:
get_full_address('3544nv', '45')

Parkzichtlaan 45  
3544NV UTRECHT


[['PARKZICHTLAAN', '45', '', '3544NV', 'UTRECHT']]

In [197]:
get_full_address('9999NV', '45')

bad address


[]

In [198]:
get_full_address('5531PE', '1')

Hulselseweg 1  
5531PE BLADEL
Hulselseweg 1 A 
5531PE BLADEL


[['HULSELSEWEG', '1', '', '5531PE', 'BLADEL'],
 ['HULSELSEWEG', '1', 'A', '5531PE', 'BLADEL']]

# From Street

In [201]:
option2 = Options()
option2.headless = True
driver2 = webdriver.Firefox(options=option2)
driver2.get('https://www.postnl.nl/en/find-a-postcode/')

In [212]:
def get_address_from_street(street, number, city, suffix=''):
    driver2.find_element(By.ID, 'street').clear()
    driver2.find_element(By.ID, 'number').clear()
    driver2.find_element(By.ID, 'addition').clear()
    driver2.find_element(By.ID, 'city').clear()

    driver2.find_element(By.ID, 'street').send_keys(street)
    driver2.find_element(By.ID, 'number').send_keys(number)
    driver2.find_element(By.ID, 'addition').send_keys(suffix)
    driver2.find_element(By.ID, 'city').send_keys(city)
    l = driver2.find_element(By.XPATH, "//input[@type='submit' and @value='Find postcode']")
    driver2.execute_script("arguments[0].click();", l)
    
    sleep(1)
    
    result = driver2.find_elements(By.XPATH, "//div[@class='result']")
    ans = []
    for r in result:
        try:
            street, h_number, toeg, pc, stad = (re.compile("<p>(\w+)<span class=\"house-number\">(\d+) (.*)</span></p><em>(\w+)</em><span>(\w+)</span>").match(r.get_attribute('innerHTML')).groups())
            print(street, h_number, toeg, pc, stad)
            ans.append(list(map(lambda x: x.upper(), (street, h_number, toeg, pc, stad))))
        except:
            print('bad address')
    
    return ans

In [210]:
get_address_from_street('Parkzichtlaan', '45', 'Utrecht')

Parkzichtlaan 45  3544NV UTRECHT


[['PARKZICHTLAAN', '45', '', '3544NV', 'UTRECHT']]

In [213]:
get_address_from_street('HULSELSEWEG', '1', 'BLADEL')

Hulselseweg 1  5531PE BLADEL
Hulselseweg 1 A 5531PE BLADEL


[['HULSELSEWEG', '1', '', '5531PE', 'BLADEL'],
 ['HULSELSEWEG', '1', 'A', '5531PE', 'BLADEL']]

In [214]:
get_address_from_street('HULSELSEWEG', '1', 'BLADEL', 'A')

Hulselseweg 1 A 5531PE BLADEL


[['HULSELSEWEG', '1', 'A', '5531PE', 'BLADEL']]